**1º PASO: Importación de librerías que se utilizarán en el proyecto**

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.ticker import FormatStrFormatter
import seaborn as sns
import ast
import json

# Desarrollo de la API

Siguiendo con el desarrollo del proyecto, se implementarán 6 fuciones específicas para la realización de consultas por parte de un usuario, a través de una API que será deployed posteriomente mediante **Render**.

Todas las funciones, al igual que el modelo de ML que se desarrollará para implementar un sistema de recomendación, se alimentarán con el DataFrame obtenido en la etapa de ETL (`movies_ETL.csv`)

In [3]:
df_movies = pd.read_csv('movies_ETL.csv')

## Generación de Funciones

#### 1º Películas por Idioma <br>
Esta función filtra las películas por el idioma especificado como entrada y devuelve la cantidad de películas que se estrenaron en ese idioma especificado.<br>
El idioma debe ingresarse según el código ISO 639 1. Por ejemplo para el idioma _inglés_ corresponde _en_, y para _español_, _es_.

In [4]:
def peliculas_idioma(Idioma: str):
    # Filtrar las películas por idioma
    peliculas_filtradas = df_movies[df_movies['original_language'] == Idioma]
    
    # Obtener la cantidad de películas filtradas
    cantidad_peliculas = len(peliculas_filtradas)
    
    # Generar el mensaje de retorno
    mensaje_retorno = f"Hay {cantidad_peliculas} película(s) que fueron estrenadas en {Idioma}"
    
    return mensaje_retorno


In [5]:
resultado = peliculas_idioma("en")
print(resultado)  # Imprime "Hay X cantidad de películas que fueron estrenadas en es"


Hay 32249 película(s) que fueron estrenadas en en


#### 2º Duración y Año de estreno <br>
Esta función filtra las películas por el nombre de película especificado como entrada y devuelve la duración de la película y el año de estreno.

In [6]:
def peliculas_duracion(Pelicula: str):
    # Filtrar las películas por nombre
    pelicula_filtrada = df_movies[df_movies['title'] == Pelicula]
    
    # Verificar si se encontró la película
    if len(pelicula_filtrada) == 0:
        return "No se encontró la película especificada. Verifique el nombre e ingréselo nuevamente"
    
    # Obtener la duración y el año de estreno de la película
    duracion = pelicula_filtrada['runtime'].values[0]
    anio_estreno = pelicula_filtrada['release_year'].values[0]
    
    # Generar el mensaje de retorno
    mensaje_retorno = f"{Pelicula}. Duración: {duracion}. Año de estreno: {anio_estreno}"
    
    return mensaje_retorno


In [8]:
resultado = peliculas_duracion("Star Wars")
print(resultado)  # Imprime "Toy Story. Duración: x. Año de estreno: xxxx"

Star Wars. Duración: 121.0. Año de estreno: 1977


#### 3º Franquicia <br>
Esta función filtra las películas por el nombre de franquicia especificado como entrada y devuelve: <br> - Cantidad de películas que pertenecen a la franquicia<br> - Ganancia total de la franquicia, obtenida como recaudación-inversión (revenue - budget)<br> - Ganancia promedio de la franquicia, obtenida como Ganancia total/Cantidad de películas de la franquicia

In [9]:
def franquicia(Franquicia: str):
    # Filtrar las películas por pertenecer a la franquicia especificada
    peliculas_franquicia = df_movies[df_movies['franquicia'] == Franquicia]
    
    # Verificar si se encontraron películas para la franquicia especificada
    if len(peliculas_franquicia) == 0:
        return "No se encontraron películas para la franquicia especificada."
    
    # Obtener la cantidad de películas, ganancia total y ganancia promedio
    cantidad_peliculas = len(peliculas_franquicia)
    ganancia_total = (peliculas_franquicia['revenue'] - peliculas_franquicia['budget']).sum()
    ganancia_promedio = (peliculas_franquicia['revenue'] - peliculas_franquicia['budget']).mean()
    
    # Generar el mensaje de retorno
    mensaje_retorno = f"La franquicia {Franquicia} posee {cantidad_peliculas} películas, una ganancia total de {ganancia_total}, y una ganancia promedio de {ganancia_promedio}."
    
    return mensaje_retorno

In [11]:
resultado = franquicia("Despicable Me Collection")
print(resultado)  # Imprime "La franquicia Toy Story Collection posee x películas, una ganancia total de x, y una ganancia promedio de xx."

La franquicia Despicable Me Collection posee 6 películas, una ganancia total de 3392070216.0, y una ganancia promedio de 565345036.0.


#### 4º País <br>
Esta función filtra las películas por el nombre del país especificado como entrada y devuelve la cantidad de películas que fueron producidas por ese país.

In [12]:
def peliculas_pais(Pais: str):
    # Filtrar las películas por país de producción
    películas_pais = df_movies[df_movies['países'].apply(lambda paises: Pais in paises)]
    
    # Obtener la cantidad de películas producidas en el país
    cantidad_peliculas = len(películas_pais)
    
    # Generar el mensaje de retorno
    mensaje_retorno = f"Se produjeron {cantidad_peliculas} películas en el país {Pais}."
    
    return mensaje_retorno


In [13]:
resultado = peliculas_pais("Argentina")
print(resultado)  # Imprime "Se produjeron x películas en el país United States."


Se produjeron 254 películas en el país Argentina.


#### 5º Productoras <br>
Esta función filtra las películas por el nombre de la productora especificado como entrada y devuelve:<br> - Recaudación total (revenue). <br> - Cantidad de películas que realizó

In [14]:
def productoras_exitosas(Productora: str):
    # Filtrar las películas por productora
    películas_productora = df_movies[df_movies['productoras'].apply(lambda compañías: Productora in compañías)]
    
    # Calcular la recaudación total
    recaudación_total = películas_productora['revenue'].sum()
    
    # Obtener la cantidad de películas producidas
    cantidad_peliculas = len(películas_productora)
    
    # Generar el mensaje de retorno
    mensaje_retorno = f"La productora {Productora} ha tenido una recaudación total de {recaudación_total}, con un total de {cantidad_peliculas} películas producidas."
    
    return mensaje_retorno


In [15]:
resultado = productoras_exitosas("Columbia Pictures")
print(resultado)  # Imprime "La productora Universal Pictures ha tenido una recaudación total de x, con un total de x películas producidas."


La productora Columbia Pictures ha tenido una recaudación total de 43533133740.0, con un total de 901 películas producidas.


#### 6º Director <br>
Esta función filtra las películas por el nombre del director especificado como entrada y devuelve:<br> - Recaudación total (revenue). <br> - Nombre de cada película, con la fecha de lanzamiento (release_date), recaudación individual, costo y ganacia, en formato de lista

In [18]:
def get_director(nombre_director):
    # Filtrar las películas por director
    películas_director = df_movies[df_movies['director'].notnull() & df_movies['director'].apply(lambda directores: isinstance(directores, str) and nombre_director in directores)]

    # Calcular el retorno total del director
    retorno_total = películas_director['revenue'].sum()
    
    # Generar una lista de información detallada de cada película
    películas_info = []
    for index, row in películas_director.iterrows():
        película = {
            'nombre': row['title'],
            'fecha_lanzamiento': row['release_date'],
            'retorno': row['revenue'],
            'costo': row['budget'],
            'ganancia': row['revenue'] - row['budget']
        }
        películas_info.append(película)
    
    # Generar el mensaje de retorno
    mensaje_retorno = f"El director {nombre_director} ha logrado un retorno de {retorno_total}."
    
    return mensaje_retorno, películas_info


In [19]:
retorno, películas = get_director("Steven Spielberg")
print(retorno)      # Imprime "El director Steven Spielberg ha logrado un retorno de x."
print(películas)    # Imprime la lista de películas dirigidas por Steven Spielberg con su información detallada.

El director Steven Spielberg ha logrado un retorno de 9256621422.0.
[{'nombre': 'Jurassic Park', 'fecha_lanzamiento': '1993-06-11', 'retorno': 920100000.0, 'costo': 63000000, 'ganancia': 857100000.0}, {'nombre': "Schindler's List", 'fecha_lanzamiento': '1993-11-29', 'retorno': 321365567.0, 'costo': 22000000, 'ganancia': 299365567.0}, {'nombre': 'E.T. the Extra-Terrestrial', 'fecha_lanzamiento': '1982-04-03', 'retorno': 792965326.0, 'costo': 10500000, 'ganancia': 782465326.0}, {'nombre': 'Raiders of the Lost Ark', 'fecha_lanzamiento': '1981-06-12', 'retorno': 389925971.0, 'costo': 18000000, 'ganancia': 371925971.0}, {'nombre': 'Indiana Jones and the Last Crusade', 'fecha_lanzamiento': '1989-05-24', 'retorno': 474171806.0, 'costo': 48000000, 'ganancia': 426171806.0}, {'nombre': 'Jaws', 'fecha_lanzamiento': '1975-06-18', 'retorno': 470654000.0, 'costo': 7000000, 'ganancia': 463654000.0}, {'nombre': 'The Lost World: Jurassic Park', 'fecha_lanzamiento': '1997-05-23', 'retorno': 229074524.0,